In [1]:
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from pandas.core import datetools

import warnings

warnings.filterwarnings('ignore')
get_ipython().magic('matplotlib inline')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6) 

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# feature 24개

- fecha_dato : The table is partitioned for this column

- ncodpers : Customer code

- ind_empleado : Employee index A : active, B : ex-employed, F : filial, N : not employee, P : pasive #직원 여부

- pais_residencia : Customer's Country residence #거주지

- sexo : Customer's sex #성별

- age : Age #나이

- fecha_alta : The date in which the customer became as the first holder of a contract in the bank #처음 가입 날짜

- ind_nuevo : New customer Index. 1 if the customer registered in the last 6 months. #최근 6개월 가입 여부(신규가입자)

- antiguedad : Customer seniority (in months) #유지 개월 수

- indrel : 1 (First/Primary), 99 (Primary customer during the month but not at the end of the month) #주고객층 여부

- ult_fec_cli_1t : Last date as primary customer (if he isn't at the end of the month) #주고객층이었던 마지막 날짜

- indrel_1mes : Customer type at the beginning of the month ,1 (First/Primary customer), 2 (co-owner),P (Potential),3 (former primary), 4(former co-owner) #고객 타입(주고객, 공동소유자, 잠재고객, 이전 주고객, 이전 공동소유자)

- tiprel_1mes : Customer relation type at the beginning of the month, A (active), I (inactive), P (former customer),R (Potential) #월초 고객 타입

- indresi : Residence index (S (Yes) or N (No) if the residence country is the same than the bank country) #거주지 인덱스 : 거주 국가와 은행 국가의 일치 여부

- indext : Foreigner index (S (Yes) or N (No) if the customer's birth country is different than the bank country) #외국인 인덱스 : 출생 국가와 은행 국가의 일치 여부

- conyuemp : Spouse index. 1 if the customer is spouse of an employee #배우자 인덱스 : 직원의 배우자인 경우

- canal_entrada : channel used by the customer to join #가입한 채널

- indfall : Deceased index. N/S #사망 여부

- tipodom : Addres type. 1, primary address #주소 타입 : 1일 경우 주요 주소

- cod_prov : Province code (customer's address) #거주지 지역 코드

- nomprov : Province name #지역 이름

- ind_actividad_cliente : Activity index (1, active customer; 0, inactive customer) #활동 인덱스 : 1, 0

- renta : Gross income of the household #총 수입

- segmento : segmentation: 01 - VIP, 02 - Individuals 03 - college graduated #구분 : VIP / 개인 / 대학졸업자

# Class 24개

- ind_ahor_fin_ult1 : Saving Account #저축통장
- ind_aval_fin_ult1 : Guarantees #보증
- ind_cco_fin_ult1 : Current Accounts #예금
- ind_cder_fin_ult1 : Derivada Account #파생계좌
- ind_cno_fin_ult1	Payroll Account #월급통장
- ind_ctju_fin_ult1 : Junior Account #어린이계좌
- ind_ctma_fin_ult1	: Más particular Account #특별계좌1
- ind_ctop_fin_ult1 : particular Account #특별계좌2
- ind_ctpp_fin_ult1 : particular Plus Account #특별계좌3
- ind_deco_fin_ult1 : Short-term deposits #단기예금
- ind_deme_fin_ult1 : Medium-term deposits #중기예금
- ind_dela_fin_ult1 : Long-term deposits #장기예금
- ind_ecue_fin_ult1	: e-account #온라인계좌
- ind_fond_fin_ult1	: Funds #펀드
- ind_hip_fin_ult1 : Mortgage #담보대출
- ind_plan_fin_ult1 : Pensions #연금 
- ind_pres_fin_ult1 : Loans #대출
- ind_reca_fin_ult1 : Taxes #세금 
- ind_tjcr_fin_ult1 : Credit Card #신용카드
- ind_valo_fin_ult1 : Securities #증권
- ind_viv_fin_ult1 : Home Account #내집마련계좌
- ind_nomina_ult1 : Payroll #월급
- ind_nom_pens_ult1 : Pensions #연금
- ind_recibo_ult1 : Direct Debit #자동이체

In [ ]:
type(1.3e5)
print(1.3e5)
print(type(130000))

130000.0
<class 'int'>


In [ ]:
%time
# limit_rows = 1000000
train = pd.read_csv("train.csv", dtype={"sexo":str, "ind_nuevo":str, 
                                  "ult_fec_cli_1t":str, 
                                  "indext":str})
#, nrows=limit_rows

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10 µs


In [ ]:
train.ncodpers = train.ncodpers.astype(int)

In [ ]:
#Returns a random sample of items from an axis of object.
unique_ids = pd.Series(train["ncodpers"].unique()) #ncodpers : 고객넘버
unique_id = unique_ids.sample(n=130000)
train = train[train.ncodpers.isin(unique_id)]

In [ ]:
train["ncodpers"].count()

In [ ]:
#pandas.to_numeric(arg, errors='raise', downcast=None)
#If ‘coerce’, then invalid parsing will be set as NaN
train["age"] = pd.to_numeric(train["age"], errors="coerce")
train["antiguedad"] = pd.to_numeric(train["antiguedad"], errors="coerce") 
train["indrel_1mes"] = pd.to_numeric(train["indrel_1mes"], errors="coerce") 

In [ ]:
train["age"].dtypes

### Missing Data

In [ ]:
train.isnull().sum()

In [ ]:
train = train.drop(["ult_fec_cli_1t", "conyuemp"], axis=1)

In [ ]:
train["renta"] = train["renta"].transform(lambda x: x.fillna(x.mean()))

In [ ]:
train[train["antiguedad"] < 0] = 0

In [ ]:
#total num of products
train["tot_products"] = train.loc[:,"ind_ahor_fin_ult1":"ind_recibo_ult1"].sum(axis=1)
train["tot_products"] = pd.to_numeric(train["tot_products"], errors="coerce") 

In [ ]:
train['pais_residencia'].describe()

In [ ]:
train['ind_empleado'].value_counts()

In [ ]:
#for visualization, remove minority
train = train.loc[lambda train: train.pais_residencia == "ES", :]
train = train.loc[lambda train: train.ind_empleado == "N", :]

In [ ]:
train = train.dropna(axis=0)

In [ ]:
train.count(0)

In [ ]:
train.ind_actividad_cliente = train.ind_actividad_cliente.astype(int)

### test data

In [ ]:
%time
test = pd.read_csv("test.csv", dtype={"sexo":str, "ind_nuevo":str, 
                                  "ult_fec_cli_1t":str, 
                                  "indext":str})

In [ ]:
test.shape

In [ ]:
test["age"] = pd.to_numeric(train["age"], errors="coerce")
test["antiguedad"] = pd.to_numeric(train["antiguedad"], errors="coerce") 


In [ ]:
test.isnull().sum()

In [ ]:
test["canal_entrada"].mode()

In [ ]:
test["sexo"] = test["sexo"].transform(lambda x: x.fillna('V'))

In [ ]:
test["segmento"] = test["segmento"].transform(lambda x: x.fillna('02 - PARTICULARES'))

In [ ]:
test["cod_prov"] = test["cod_prov"].transform(lambda x: x.fillna('28.0'))

In [ ]:
test["nomprov"] = test["nomprov"].transform(lambda x: x.fillna('MADRID'))

In [ ]:
test["canal_entrada"] = test["canal_entrada"].transform(lambda x: x.fillna('KHE'))

In [ ]:
test.ind_actividad = test.ind_actividad_cliente.astype(int)

In [ ]:
test.isnull().sum()

### 변수 선택

In [ ]:
train_x = train[['sexo', 'renta', 'ind_nuevo', 'cod_prov', 'segmento', 'ind_actividad_cliente']]

In [ ]:
test_x = test[['sexo', 'renta', 'ind_nuevo', 'cod_prov', 'segmento', 'ind_actividad_cliente']]

- 클래스 개수

In [ ]:
train_y = train[['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']]

train_count = train_y.sum()
train_count = train_count.drop(train_count[train_count == 0].index).sort_values(ascending=False)[:30]
count_data = pd.DataFrame({'count' :train_count})
count_data.head(24)

In [ ]:
train_x.tail()

In [ ]:
train_xd = pd.get_dummies(train_x)
train_xd.tail()

In [ ]:
train_xd.columns

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
%time
tree = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0).fit(train_xd, train_y)

In [ ]:
import io
import pydot
from IPython.core.display import Image 
from sklearn.tree import export_graphviz

def draw_decision_tree(model):
    dot_buf = io.StringIO() 
    export_graphviz(model, out_file=dot_buf, feature_names=['renta', 'cod_prov', 'ind_actividad_cliente', 'sexo_H', 'sexo_V', 'ind_nuevo_ 0', 'ind_nuevo_ 1', 'segmento_01 - TOP', 'segmento_02 - PARTICULARES', 'segmento_03 - UNIVERSITARIO'])
    graph = pydot.graph_from_dot_data(dot_buf.getvalue())[0] 
    image = graph.create_png()
    return Image(image)

In [ ]:
test_xd = pd.get_dummies(test_x)
test_xd.tail()

In [ ]:
test_y = tree.predict(test_xd)
test_y

In [ ]:
submission = pd.read_csv("sample_submission.csv")

submission["added_products"] = test_y

print(submission.shape)
submission.head()

In [ ]:
submission.to_csv("1_submission.csv", index=False)